In [ ]:
import boto3
from botocore.exceptions import ClientError

# Initialize the DynamoDB resource
dynamodb = boto3.resource("dynamodb", region_name="us-east-1")

# Define the table name
table_name = "fmbench-orchestrator"

import boto3
import logging
from botocore.exceptions import ClientError, NoCredentialsError, PartialCredentialsError

logger = logging.getLogger(__name__)


def get_or_create_table():
    try:
        # Initialize DynamoDB resource
        dynamodb = boto3.resource("dynamodb")

        # Try to load the table (will raise an exception if it doesn't exist)
        table = dynamodb.Table(table_name)
        table.load()  # Will raise a ResourceNotFoundException if the table doesn't exist
        logger.info(f"Table '{table_name}' already exists.")
    except (NoCredentialsError, PartialCredentialsError):
        # If IAM permissions are not available, return None
        logger.warning(
            "IAM permissions are not enabled or incomplete. Skipping table creation."
        )
        return None
    except ClientError as e:
        if e.response["Error"]["Code"] == "ResourceNotFoundException":
            # Table does not exist; create it
            logger.info(f"Table '{table_name}' does not exist. Creating it now...")
            table = dynamodb.create_table(
                TableName=table_name,
                KeySchema=[{"AttributeName": "UID", "KeyType": "HASH"}],
                AttributeDefinitions=[{"AttributeName": "UID", "AttributeType": "S"}],
                BillingMode="PAY_PER_REQUEST", 
            )
            table.wait_until_exists()
            logger.info(f"Table '{table_name}' has been created.")
        else:
            # Rethrow if another error occurred
            logger.error(f"Unexpected error: {e}")
            raise e

    return table


# Use the function
table = get_or_create_table()
print(f"Using table '{table_name}'.")


Table 'fmbench-orchestrator' already exists.
Using table 'fmbench-orchestrator'.


In [ ]:
from datetime import datetime


def insert_item(uid, state, instance_type, fmbench_config_file, account_id, num):
    item = {
        "uid": uid,
        "state": state,
        "instance_type": instance_type,
        "fmbench_config_file": fmbench_config_file,
        "timestamp": datetime.now().isoformat(),
        "account_id": account_id,
        "date": datetime.now().strftime("%Y-%m-%d"),
        "num": num,
    }

    table.put_item(Item=item)
    print(f"Inserted item: {item}")


insert_item(
    uid="10220",
    state="start",
    instance_type="g5.2xl",
    fmbench_config_file="g5-llama3",
    account_id="account_id",
    num=1,
)


Inserted item: {'uid': '10220', 'state': 'start', 'instance_type': 'g5.2xl', 'fmbench_config_file': 'g5-llama3', 'timestamp': '2024-11-09T14:14:30.717206', 'account_id': 'account_id', 'date': '2024-11-09', 'num': 1}


In [ ]:
def get_all_items():
    try:
        response = table.scan()
        items = response["Items"]

        # Keep scanning if there are more items
        while "LastEvaluatedKey" in response:
            response = table.scan(ExclusiveStartKey=response["LastEvaluatedKey"])
            items.extend(response["Items"])

        print("All items in table:")
        for item in items:
            print(item)
        return items
    except Exception as e:
        print(f"Error fetching items: {e}")


# Example usage
all_items = get_all_items()


All items in table:
{'date': '2024-11-09', 'instance_type': 'g5.2xl', 'num': Decimal('1'), 'timestamp': '2024-11-09T14:14:30.717206', 'account_id': 'account_id', 'fmbench_config_file': 'g5-llama3', 'uid': '10220', 'state': 'start'}
